In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances

Data Loading

In [2]:
ratings_train = pd.read_csv("./Data/rate_train.csv", low_memory=False)
ratings_test = pd.read_csv("./Data/rate_test.csv",low_memory=False)

In [3]:
datasets_train = ratings_train.dataset_id.unique()
model_train = ratings_train.model_id.unique()
datasets_test = ratings_test.dataset_id.unique()
model_test = ratings_test.model_id.unique()
meta_models = pd.read_csv("./Data/models_num.csv",low_memory=False)
models = meta_models.model_id.unique()

Rating Matrix

In [4]:
data_model_train_matrix = pd.DataFrame(index=datasets_train,columns=models)
data_model_test_matrix = pd.DataFrame(index=datasets_test,columns=model_test)

In [5]:
for row in ratings_train.itertuples():
    data_model_train_matrix.loc[row[1]][row[2]] = row[3]


In [6]:
for row in ratings_test.itertuples():
    data_model_test_matrix.loc[row[1]][row[2]] = row[3]
data_model_test_matrix = data_model_test_matrix.fillna(0)

Dataset Similarity Matrix

In [7]:
meta_datasets = pd.read_csv("./Data/datasets_v.csv",low_memory=False)
datasets = meta_datasets.data_id.unique()
meta_datasets = meta_datasets.loc[:,("v1","v2","v3","v4","v5","v6","v7","v8")]
scaler = StandardScaler()

# 对dataframe的数据进行标准化
scaled_data = scaler.fit_transform(meta_datasets)
# 将标准化后的数据转换为dataframe，并保留原始索引
scaled_df = pd.DataFrame(scaled_data, index=meta_datasets.index, columns=meta_datasets.columns)
meta_dataset_similarity = cosine_similarity(scaled_df.values.tolist())
meta_dataset_similarity = pd.DataFrame(meta_dataset_similarity,index=datasets,columns=datasets)

KNN sim

In [8]:
meta_datasets

,v1,v2,v3,v4,v5,v6,v7,v8
0,0.454312,0.241966,-0.178474,0.078718,-0.343706,0.150178,0.103396,-0.474233
1,0.527839,0.284245,-0.201757,0.116918,-0.394861,0.168884,0.137184,-0.536491
2,0.421102,0.234788,-0.172241,0.081514,-0.348054,0.138399,0.100450,-0.447096
3,0.358012,0.198786,-0.129784,0.068405,-0.275457,0.119678,0.093582,-0.378495
4,0.298305,0.167896,-0.120117,0.052906,-0.242469,0.107290,0.072812,-0.322821
...,...,...,...,...,...,...,...,...
67,0.446627,0.241174,-0.179276,0.092747,-0.355247,0.140657,0.110482,-0.472832
68,0.411207,0.244925,-0.156990,0.084651,-0.319750,0.147412,0.104277,-0.440762
69,0.495822,0.270633,-0.181879,0.100016,-0.387360,0.150677,0.108678,-0.507951
70,0.406016,0.214886,-0.155250,0.082475,-0.315015,0.126135,0.087693,-0.414519


In [9]:
def cosine_similarity_func(ratings, user1, user2):
    # 找到两个用户共同评分的物品，并将这些评分放入一个向量中
    u1_ratings = ratings.loc[user1].dropna()
    u2_ratings = ratings.loc[user2].dropna()

    common_items = np.intersect1d(u1_ratings.index, u2_ratings.index).tolist()
    u1_common_ratings = u1_ratings.loc[common_items]
    u2_common_ratings = u2_ratings.loc[common_items]

    # 计算两个向量之间的余弦相似度
    if len(common_items) == 0:
        return 0
    else:
        cos_sim = np.dot(u1_common_ratings, u2_common_ratings) / (np.linalg.norm(u1_common_ratings) * np.linalg.norm(u2_common_ratings))
        return cos_sim

In [10]:
dataset_similarity = pd.DataFrame(index=datasets_train,columns=datasets_train)

In [11]:
def create_bipartite_adjacency_matrix(rating_matrix):
    n_users, n_items = rating_matrix.shape
    adjacency_matrix = np.zeros((n_users + n_items, n_users + n_items))
    adjacency_matrix[:n_users, n_users:] = rating_matrix
    adjacency_matrix[n_users:, :n_users] = rating_matrix.T
    return adjacency_matrix

In [12]:
def propagation_matrix(adjacency, lambda_):
    n = adjacency.shape[0]
    I = np.eye(n)
    # 将 NaN 视为 0
    adjacency = np.nan_to_num(adjacency)
    try:
        P = np.linalg.inv(I - lambda_ * adjacency)
    except np.linalg.LinAlgError:
        print("矩阵不可逆，无法计算传播矩阵")
        return None
    return P

In [13]:
def propagation_matrix_withWalkLength(adjacency_matrix, max_walk_length):
    adjacency_matrix = np.nan_to_num(adjacency_matrix)
    propagation_matrix = np.eye(adjacency_matrix.shape[0])
    sum_matrix = np.eye(adjacency_matrix.shape[0])

    for _ in range(max_walk_length):
        propagation_matrix = propagation_matrix @ adjacency_matrix
        sum_matrix += propagation_matrix

    return sum_matrix

In [14]:
start_time_train = time.time()

In [15]:
# 计算二分图邻接矩阵
bipartite_adjacency_matrix = create_bipartite_adjacency_matrix(data_model_train_matrix)

In [16]:
propagation_maxLength = propagation_matrix_withWalkLength(bipartite_adjacency_matrix, 6)

In [17]:
# 提取用户-商品传播矩阵和商品-用户传播矩阵
n_users = data_model_train_matrix.shape[0]
user_item_propagation = propagation_maxLength[:n_users, n_users:]
item_user_propagation = propagation_maxLength[n_users:, :n_users]

In [18]:
# 计算 Random Walk Kernel
random_walk_kernel = np.dot(user_item_propagation, item_user_propagation)

In [19]:
def normalize_kernel(kernel_matrix):
    diagonal_elements = np.diag(kernel_matrix)
    normalized_kernel_matrix = np.divide(kernel_matrix, np.sqrt(np.outer(diagonal_elements, diagonal_elements)))
    return normalized_kernel_matrix

In [20]:
normalized_kernel = normalize_kernel(random_walk_kernel)

In [21]:
normalized_kernel = pd.DataFrame(normalized_kernel)

In [22]:
normalized_kernel

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,1.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.999381,0.00000,0.0,0.000000
1,0.000000,1.00000,0.0,0.000000,0.0,0.0,0.000000,1.000000,0.000000,0.999962,...,1.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.99996,0.0,0.000000
2,0.000000,0.00000,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.999115,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000
3,0.000000,0.00000,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.997179
4,0.000000,0.00000,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.999998,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,1.0,0.000000,0.00000,0.0,0.000000
68,0.999381,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.00000,0.0,0.000000
69,0.000000,0.99996,0.0,0.000000,0.0,0.0,0.000000,0.999961,0.000000,1.000000,...,0.999961,0.0,0.0,0.0,0.0,0.0,0.000000,1.00000,0.0,0.000000
70,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,1.0,0.000000


In [23]:
lambda_ = 0.5
for i in datasets_train:
    for j in datasets_train:
        # rating_based_sim = cosine_similarity_func(data_model_train_matrix,i,j)
        if normalized_kernel.loc[i][j] != 0 and meta_dataset_similarity.loc[i][j] != 0:
            dataset_similarity.loc[i][j] = lambda_ * normalized_kernel.loc[i][j] + (1-lambda_) * meta_dataset_similarity.loc[i][j]
            continue
        dataset_similarity.loc[i][j] = 0


In [24]:
dataset_similarity

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.712598,0,0,0
1,0,1.0,0,0,0,0,0,0.966721,0,0.025001,...,0.880545,0,0,0,0,0,0,0.953382,0,0
2,0,0,1.0,0,0,0,0,0,0.496074,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.974817
4,0,0,0,0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0,0,0,0,0,0,0.964098,0,0,0,...,0,0,0,0,0,1.0,0,0,0,0
68,0.712598,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.0,0,0,0
69,0,0.953382,0,0,0,0,0,0.924301,0,0.109346,...,0.740674,0,0,0,0,0,0,1.0,0,0
70,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,0


Prediction

In [25]:
def predict_ratings(rating_matrix, user_similarity_matrix, k=5):
    """
    输入：
    rating_matrix - 评分矩阵，DataFrame格式，其中NaN表示未评分
    user_similarity_matrix - 用户相似度矩阵，DataFrame格式
    k - 最近邻的数量，默认为5

    输出：
    prediction_matrix - 预测矩阵，DataFrame格式
    """

    # 初始化预测矩阵
    prediction_matrix = rating_matrix.copy()

    # 对于评分矩阵中的每个NaN值，使用K最近邻的方法预测评分
    for i in range(rating_matrix.shape[0]):
        for j in range(rating_matrix.shape[1]):
            if np.isnan(rating_matrix.iloc[i, j]):
                # 获取第i个用户的相似度值，并在相似度矩阵中找到K个最相似的用户
                similarity_values = user_similarity_matrix.iloc[i].sort_values(ascending=False)[1:k+1]

                # 计算加权平均评分
                weighted_sum = 0
                similarity_sum = 0
                for index, value in similarity_values.items():
                    user_rating = rating_matrix.iloc[index, j]
                    if not np.isnan(user_rating):
                        weighted_sum += value * user_rating
                        similarity_sum += value

                # 如果存在至少一个相似用户对该物品进行了评分，则计算预测评分
                if similarity_sum != 0:
                    prediction_matrix.iloc[i, j] = weighted_sum / similarity_sum
                else:
                    # 如果没有相似用户评分，则使用当前用户的平均评分作为预测值
                    prediction_matrix.iloc[i, j] = rating_matrix.iloc[i].mean()

    return prediction_matrix

In [26]:
def predict(rating_matrix, similarity_matrix):
    """
    根据评分矩阵和相似度矩阵预测评分。

    参数：
    rating_matrix (pd.DataFrame)：评分矩阵，包含NaN值
    similarity_matrix (numpy.array)：相似度矩阵

    返回：
    pd.DataFrame：预测评分矩阵
    """

    # 获取评分矩阵的均值（忽略NaN值）
    mean_rating = rating_matrix.mean(axis=1).values

    # 将评分矩阵中的NaN值替换为0
    rating_matrix_nan_to_zero = rating_matrix.fillna(0).values

    # 减去均值，得到归一化的评分矩阵
    normalized_rating_matrix = rating_matrix_nan_to_zero - mean_rating[:, np.newaxis]

    # 计算预测评分
    predicted_ratings = mean_rating[:, np.newaxis] + np.dot(similarity_matrix, normalized_rating_matrix) / np.abs(similarity_matrix).sum(axis=1)[:, np.newaxis]

    # 将预测评分数组转换为DataFrame
    predicted_ratings_df = pd.DataFrame(predicted_ratings, index=rating_matrix.index, columns=rating_matrix.columns)

    return predicted_ratings_df

In [27]:
model_prediction_train = predict(data_model_train_matrix,dataset_similarity)
model_prediction_train = pd.DataFrame(model_prediction_train,index=datasets_train,columns=models).sort_index().sort_index(axis=1)

c:\users\byy\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


In [28]:
model_prediction_train

,72,73,74,75,76,77,78,79,80,81,...,1862,1863,1864,1865,1866,1867,1868,1869,1870,1871
0,0.940459,0.958896,0.958319,0.99972,0.999439,0.997482,0.999439,0.99972,0.998881,0.999439,...,0.052126,0.052126,0.052126,0.052126,0.052126,0.052126,0.052126,0.052126,0.052126,0.052126
1,0.014812,0.014812,0.014812,0.014812,0.014812,0.014812,0.014812,0.014812,0.014812,0.014812,...,0.014812,0.014812,0.014812,0.014812,0.014812,0.014812,0.014812,0.014812,0.014812,0.014812
2,-0.030882,-0.030882,-0.030882,-0.030882,-0.030882,-0.030882,-0.030882,-0.030882,-0.030882,-0.030882,...,-0.030882,-0.030882,-0.030882,-0.030882,-0.030882,-0.030882,-0.030882,-0.030882,-0.030882,-0.030882
3,-0.005004,-0.005004,-0.005004,-0.005004,-0.005004,-0.005004,-0.005004,-0.005004,-0.005004,-0.005004,...,0.165514,0.144654,0.142857,0.162836,0.142857,0.143773,0.142857,0.133342,0.144218,0.140705
4,-0.036939,-0.036939,-0.036939,-0.036939,-0.036939,-0.036939,-0.036939,-0.036939,-0.036939,-0.036939,...,-0.036939,-0.036939,-0.036939,-0.036939,-0.036939,-0.036939,-0.036939,-0.036939,-0.036939,-0.036939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.013501,0.013501,0.013501,0.013501,0.013501,0.013501,0.013501,0.013501,0.013501,0.013501,...,0.013501,0.013501,0.013501,0.013501,0.013501,0.013501,0.013501,0.013501,0.013501,0.013501
68,0.350863,0.358772,0.339103,0.376309,0.333301,0.36778,0.325719,0.280354,0.34698,0.308344,...,-0.071981,-0.071981,-0.071981,-0.071981,-0.071981,-0.071981,-0.071981,-0.071981,-0.071981,-0.071981
69,-0.032124,-0.032124,-0.032124,-0.032124,-0.032124,-0.032124,-0.032124,-0.032124,-0.032124,-0.032124,...,-0.032124,-0.032124,-0.032124,-0.032124,-0.032124,-0.032124,-0.032124,-0.032124,-0.032124,-0.032124
70,-0.010128,-0.010128,-0.010128,-0.010128,-0.010128,-0.010128,-0.010128,-0.010128,-0.010128,-0.010128,...,-0.010128,-0.010128,-0.010128,-0.010128,-0.010128,-0.010128,-0.010128,-0.010128,-0.010128,-0.010128


In [29]:
model_prediction_test = pd.DataFrame(index=datasets_test,columns=model_test)

In [30]:
end_time_train = time.time()
Training_time = end_time_train - start_time_train
Training_time

1.7108173370361328

In [31]:
def find_sim_index(index):
    row1 = meta_dataset_similarity.loc[index]
    row1_max_index = row1[row1 == row1.max()].index[0]
    return row1_max_index

In [32]:
def Find_Top_k(i,sim_matrix):
    row = sim_matrix.loc[i]
    row = row.sort_values(ascending=False)
    index_row = row.index
    index_row = index_row.values.tolist()
    return index_row

In [33]:
start_time = time.time()

In [34]:
for dataset in datasets_test:
    for model in model_test:
        dataset_sim_list = Find_Top_k(dataset,meta_dataset_similarity)[1:]
        # 仅保留存在于 model_prediction_train 的索引
        valid_indices = [idx for idx in dataset_sim_list if idx in model_prediction_train.index][:15]
        model_prediction_test.loc[dataset][model] = model_prediction_train.loc[valid_indices][model].mean()

In [35]:
end_time = time.time()

In [36]:
end_time - start_time

4.886613130569458

In [37]:
for i in datasets_test:
    for j in model_test:
        if data_model_test_matrix.loc[i][j] == 0:
            model_prediction_test.loc[i][j] = None

In [38]:
result = pd.DataFrame(columns={"dataset","model","balanced_accuracy","groundtruth_balanced_accuracy"})

In [39]:
for i in datasets_test:
    for j in model_test:
        if model_prediction_test.loc[i][j] is not None:
            balanced_accuracy = model_prediction_test.loc[i][j]
            groundtruth_balanced_accuracy = data_model_test_matrix.loc[i][j]
            result = result.append([{'dataset':i,'model':j,'balanced_accuracy':balanced_accuracy,'groundtruth_balanced_accuracy':groundtruth_balanced_accuracy}],ignore_index=True)

In [40]:
result

,dataset,groundtruth_balanced_accuracy,model,balanced_accuracy
0,1,0.672734,97,0.390627
1,1,0.748037,98,0.426927
2,1,0.755532,99,0.418527
3,1,0.995360,100,0.592627
4,1,0.994290,101,0.593127
...,...,...,...,...
1370,67,0.500000,1767,0.193562
1371,67,0.500000,1768,0.193844
1372,67,0.577401,1769,0.212610
1373,67,0.500000,1770,0.195642


In [41]:
result.to_csv("../Kaggle/Output/Dataset_RandomWalk/Full_Dataset_RandomWalk@6@15.csv",index=False)